In [7]:
from pandas import read_csv 
from datetime import datetime

data_set = read_csv('mudac_master.csv')
data_set = data_set.iloc[:data_set['date'].last_valid_index(,:] # Truncate dataset to those where every data is available 
first_col


TypeError: 'RangeIndex' object is not callable